In [3]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 43.3 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 39.9 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845515 sha256=6fc5aaddb091ceecd84f5c798d3f1bf06e2e1ce75d9ae8a1d9f9a44bb764a2d0
  Stored in directory: /Users/itong1900/Library/Caches/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [5]:
%%time
spark = SparkSession.builder.appName("ottoDB").getOrCreate()

CPU times: user 14.5 ms, sys: 18.5 ms, total: 33 ms
Wall time: 6.69 s


In [60]:
%%time
trainDf = spark.read.json("../../train.jsonl",lineSep='\n')
testDf = spark.read.json("../../test.jsonl",lineSep='\n')

CPU times: user 4.03 ms, sys: 4.56 ms, total: 8.59 ms
Wall time: 19.1 s


In [10]:
trainDf.printSchema()

root
 |-- events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aid: long (nullable = true)
 |    |    |-- ts: long (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- session: long (nullable = true)



In [41]:
def convert_type_num(arr):
    return [0 if i == "clicks" else (1 if i == "carts" else 2) for i in arr]
convert_type_num_udf = udf(lambda row: convert_type_num(row))

In [61]:
testDf_transformed = testDf.withColumn("total_action", size(col("events")))\
                          .withColumn("aids", col("events.aid"))\
                          .withColumn("ts", col("events.ts"))\
                          .withColumn("type", convert_type_num_udf(col("events.type")))\
                          .select("session", "total_action", "aids", "ts", "type")  

trainDf_transformed = trainDf.withColumn("total_action", size(col("events")))\
                          .withColumn("aids", col("events.aid"))\
                          .withColumn("ts", col("events.ts"))\
                          .withColumn("type", convert_type_num_udf(col("events.type")))\
                          .select("session", "total_action", "aids", "ts", "type")  

In [62]:
trainDf_transformed.show(5)

+-------+------------+--------------------+--------------------+--------------------+
|session|total_action|                aids|                  ts|                type|
+-------+------------+--------------------+--------------------+--------------------+
|      0|         276|[1517085, 1563459...|[1659304800025, 1...|[0, 0, 0, 0, 0, 0...|
|      1|          32|[424964, 1492293,...|[1659304800025, 1...|[1, 0, 1, 0, 1, 0...|
|      2|          33|[763743, 137492, ...|[1659304800038, 1...|[0, 0, 0, 0, 0, 0...|
|      3|         226|[1425967, 1425967...|[1659304800095, 1...|[1, 0, 0, 0, 1, 0...|
|      4|          19|[613619, 298827, ...|[1659304800119, 1...|[0, 0, 2, 0, 0, 0...|
+-------+------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [63]:
%%time
trainDf_transformed.count()

CPU times: user 1.42 ms, sys: 1.28 ms, total: 2.7 ms
Wall time: 8.62 s


12899779

In [64]:
%%time
testDf_transformed.count()

CPU times: user 609 µs, sys: 985 µs, total: 1.59 ms
Wall time: 704 ms


1671803

In [ ]:
trainDf_transformed.coalesce(1).write.format("parquet").save("../../data/train.parquet")

In [66]:
testDf_transformed.coalesce(1).write.format("parquet").save("../../data/test.parquet")